In [2]:
import requests as req
from bs4 import BeautifulSoup as bsp
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from seleniumwire import webdriver  # blinker == 1.7.0
import requests
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from fake_useragent import UserAgent
from googlesearch import search

base = 'https://www.amazon.com'
thirdWheel = '/s?k='

In [ ]:
# session = requests.Session()
# session.headers.update({"User-Agent": UserAgent().random})
driver = webdriver.Chrome()
# driver = webdriver.Firefox(service = service, options = options)
driver.get(base)
#   It will scroll to right above the footer of the page then scoll to the top then back to the bottom untill there is no new items being loaded
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(10) 
html = driver.page_source
driver.quit()          
soup = bsp(html, 'lxml')
boxes = soup.find_all('a', class_ = 'a-link-normal _fluid-quad-image-label-v2_style_centerImage__30wh- aok-block image-window')
print(boxes)
links = []
urls = []
categ = []
for box in boxes:
    actual = box.get('href')
    url = base + actual
    print("Url : ", url)
    urls.append(url)
    som = box.get('aria-label')
    link = som.replace(' ', '+')
    link = base + thirdWheel + link
    print("Links : ", link)
    links.append(link)
    categ.append(som)
    print(som)


In [4]:
def get_valid_proxies():
    proxy_list_url = 'https://free-proxy-list.net/'
    response = requests.get(proxy_list_url)
    soup = bsp(response.text, 'html.parser')
    proxy_data = []
    rows = soup.find_all('tr')[1:]
    for row in rows:
        columns = row.find_all('td')
        if len(columns) >= 8:
            ip_address = columns[0].text.strip()
            google_enabled = columns[5].text.strip().lower() == 'yes'
            https_enabled = columns[6].text.strip().lower() == 'yes'
            last_checked = columns[7].text.strip()
            if (last_checked.endswith('mins ago') and int(last_checked.split(' ')[0]) < 15) or last_checked.endswith('hours ago'):
                if google_enabled or https_enabled:
                    proxy_data.append({'ip_address': ip_address, 'google_enabled': google_enabled, 'https_enabled': https_enabled})

    return proxy_data


In [5]:
def rotate_user_agent(proxy):
    if proxy:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'http': f'http://{proxy}',
            'https': f'https://{proxy}'
        }
    else:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    return headers

In [ ]:
products = []
categories = []
cat = 0
# proxies = get_valid_proxies()
for url in urls:
    total_samples = 0
    myurl = url
    while myurl:
        # proxy = proxies[total_samples % len(proxies)] if proxies else None
        options = {
            'headers' : {
                "User-Agent": UserAgent().random
            }
        }
        print("VISITING : ", myurl)
        # sdsd = Options()
        # sdsd.add_argument("--headless")  # Run in background
        # sdsd.add_argument("--disable-gpu")
        # sdsd.add_argument("--no-sandbox")
        # sdsd.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(seleniumwire_options = options)
        # driver = webdriver.Firefox()
        driver.get(myurl)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(10)
        html = driver.page_source
        # driver.quit()
        # headers = rotate_user_agent(proxy)
        # response = req.get(myurl, headers = headers)
        # session = requests.Session()
        # session.headers.update({"User-Agent": UserAgent().random})
        # response = req.get(myurl, headers = {"User-Agent": UserAgent().random})
        # if response.status_code == 200:
            # soup = bsp(response.text, 'lxml')
        soup = bsp(html, 'lxml')
        print(soup)
        productes = soup.find_all('a', class_ = 'a-link-normal s-line-clamp-4 s-link-style a-text-normal')
        
        if not productes:
            productes = soup.find_all('a' , class_= 'a-link-normal s-line-clamp-2 s-link-style a-text-normal')
        print(productes)
        for product in productes:
            product = base + product.get('href')
            products.append(product)
            if cat in range(len(categ)):
                categories.append(categ[cat])
            print(product)
        
        total_samples += 1
        nextP = soup.find('a', class_ = 's-pagination-item s-pagination-next s-pagination-button s-pagination-button-accessibility s-pagination-separator')
        if nextP:
            myurl = base + nextP.get('href')
        if total_samples >= 3:
            break
    cat += 1

In [ ]:
# Create a DataFrame
data = {"Products": products, "Categories": categories}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
file_name = "products.csv"
df.to_csv(file_name)

print(f"Data saved to {file_name}")
